## Motivating Example

- In Fishtown, buses do not operate on a fixed schedule. Instead, they arrive according to a Poisson process at a rate of one per 10 minutes. How long would you have to wait if you show up at a bus stop at an arbitrary time?

- Most people guess that they would have to wait about 5 minutes, since usually you will show up at the bus stop in between bus arrivals. Surprisingly, the answer is that you have to wait just as long as if you had arrived right when the previous bus was leaving! We will see why at the end of this lesson.

## Theory

### Definition

- **Definition 35.1**: As covered in chapter 33, an exponential distribution can be seen as the distribution of time between Poisson arrivals. A random variable X with a distribution of $\text{Exponential}(\lambda)$ has the following definitions:
    - PDF: $f(x) = \begin{Bmatrix} \lambda e^{-\lambda x} & x \ge 0 \\ 0 & \text{otherwise} \end{Bmatrix}$
    - CDF: $F(x) = \begin{Bmatrix} 1 - e^{-\lambda x} & x \ge 0 \\ 0 & \text{otherwise} \end{Bmatrix}$

- How do we know that the time of first arrival of a poisson process follows exponential distribution?
    - Let $T$ be the time of first arrival. For $t > 0$, CDF of $T$ is:
    - $$\begin{align} F(t) &= P(T \le t) \\ &= 1 - P(T \gt t) \\ &= 1 - P(\text{0 arrivals in (0,t)}) \\ &= 1 - e^{-\lambda t} \frac{(\lambda t)^0}{0!} \\ &= 1 - e^{-\lambda t} \end{align}$$
    - This is the CDF of an exponential variable as shown above

- **Theorem 35.2** In exponential distribution, interarrival times are independent
    - Let $T_{n}$ be the time of the n-th arrival. So across all values of time, there will be a $T_1, T_2, ...$
    - Independence of interarrival times means that $P(T_j > t | T_i = x) = P(\text{No events in [x, x+t]})$, 
        - where $x, t \in \mathbb{R} \text{ AND } x \lt t \text{ AND } j \gt i$
        - In english: For second arrival to be after t, and first arrival to be at x, it implies that (i) there are no arrivals between x and t AND (ii) this probability is independent of the value of $T_1$

    - Proof (Analytic proof): 
        - $$\begin{align}
            P(T_i > t | T_{j} = x) &= P(\text{No arrivals in window [x,x+t]} | T_j = x) \\
            &= P(\text{No arrivals in window [x,x+t]}) & \text{by independence of time periods} \\
            &= e^{-\lambda t} \frac{(\lambda t)^0}{0!} \\
            &= e^{-\lambda t}
            \end{align}$$

    - Proof (Concrete example):
        - Let's suppose the first arrival is at time 10, and second arrival is at time 11. 
        - (Probability of second arrival not between 10 and 11 GIVEN first arrival at 10) is simply (1 - probability of second arrival between 10 and 11 given first arrival by 10)
        - (probability of second arrival between 10 and 11 given first arrival by 10) is the same as (probability of second arrival between 10 and 11) because poisson arrivals are independent by definition
        - (probability of second arrival between 10 and 11) is the same as (probability of first arrival between 0 and 1) because arrivals are independent by definition of Poisson

    - Proof (Simulation)
        - Let's divide a time period into discrete blocks, and assume that arrival in each block is binomial with probability $\mu$
        - We make use of the proof from chapter 16 that this the PMF of a binomial converges to a Poisson distribution as the time periods get arbitrarily small and count of time periods approach infinity

In [2]:
import numpy as np
MU = 0.4
BINOMIAL_DRAWS = 1
TIME_PERIODS = 10
SAMPLES = 10_000
samples = [np.random.binomial(n=BINOMIAL_DRAWS, p=MU, size=TIME_PERIODS) for _ in range(SAMPLES)]

count_arrivals_t2_t3 = sum([1 if x[2] == x[1] == 1 else 0 for x in samples])
count_arrivals_t2 = sum([1 if x[1]==1 else 0 for x in samples])
count_arrivals_t3 = sum([1 if x[2]==1 else 0 for x in samples])

probability_arrival_t3 = count_arrivals_t3/SAMPLES
probability_arrival_t3_given_t2 = count_arrivals_t2_t3/count_arrivals_t2
print(probability_arrival_t3_given_t2, probability_arrival_t3)

0.39056984367120523 0.4002


### Example 35.1 (Lifetime of a bulb)

- Assume that a lifetime of a bulb is an $\text{Exponential}(0.3)$ random variable. What is the probability that it lasts between 1 and 5 years?
    - In this case, $\lambda = 0.3$, which refers to the expected number of "arrivals" (in this case, new bulbs), per year
    - At 0.3 bulbs per year, each bulb is expected to last for 1/0.3 = 3.33 years

- To get the probability that a bulb lasts between 1 and 5 years, we can: 
    - Integrate the PDF of an exponential distribution between 1 and 5
        - $\int_{1}^{5} f(X) = \int_1^5 \lambda e^{-\lambda x} \approx 0.517$
    - Get CDF of 5 - CDF of 1
        - $F(5)-F(1) = 1 - e^{-0.3 \cdot 5} - 1 + e^{-0.3 \cdot 1} \approx 0.517 $ 

### Theorem 35.3: Exponential distribution is memoryless

- Another interesting property of the exponential distribution is that it is memoryless. This means that $ P(X \gt s + t |  X \gt s) = P(X \gt t)$
    - where $X$ is a random variable denoting the time to the next arrival, and $s, t$ are constants

- Proof (Analytical)
    - $$\begin{align}
        P(X \gt s+t | X \gt s) &= \frac{P(X \gt s+t \text{ AND } X \gt s)}{P(X \gt s)} \\
        &= \frac{P(X \gt s+t)}{P(X \gt s)} & \text{X > s+t immediately means X > s} \\
        &= \frac{e^{-\lambda(s+t)}}{e^{-\lambda s}} \\
        &= e^{-\lambda t} \\
        &= P(X \gt t)
        \end{align}$$

### Example 35.2 (Waiting time paradox)

- If buses in Fishtown arrive at a bus stop according to a Poisson process at a rate of one per 10 minutes (i.e., $\lambda = 0.1$ arrivals per minute), how long do you have to wait before the next bus arrives?

    - We know from Theorem 35.2 that the time X, between when the previous bus arrived and when the next bus will arrive, follows an $\text{Exponential}(\lambda = 0.1)$ distribution. However, you showed up at the bus stop some time $s$ after the previous bus had left, so you should not have to wait as long as $X$.
    - Instead, you might think that waiting time is $W = X - s$, given that $X \gt s$ (i.e., the next bus has not arrived yet when you show up at the stop).
    - But by the memoryless property (35.3):
    - $$ P(W \gt t | X \gt s) = P(X - s \gt t | X \gt s) = P(X \gt t) $$
    - So you have to wait just as long as if you had showed up right as the previous bus was leaving. You do not save any time by showing up in between bus arrivals!

- **Explanation**
    - This is one of those examples that is mindblowing until you see it in a different way. 
        - In the proof for independence of interarrival time, we said that T is an exponential distribution that represents the time between Poisson arrivals
        - In an earlier chapter, we also noted that a Poisson distribution over a time interval can be seen as infinitely many infinitesimally small Bernoulli draws. So in every time period, there is a large chance of no arrivals, and a small chance of 1 arrival, and no chance of anything more.
        - From this "multiple Bernoulli draws" perspective, waiting time for the next bus after arriving at the bus stop is basically the same as asking "what is the probability of getting a heads in a fair coin flip, if I know that the last 10 flips were tails?"
        - Since each time period is independent, whether you arrive immediately after a "heads" (i.e. after a bus arrived) or a long time after a "heads" (i.e. a long time since no bus arrives), is irrelevant. 